In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
if not Path('.git').exists():
    %cd ..
%pwd

/home/PangeAI/Documents/work/scalexa/pangeaai/optimize-cosine


'/home/PangeAI/Documents/work/scalexa/pangeaai/optimize-cosine'

# Load data

In [8]:
from cudams.utils import \
    argbatch, mkdir, get_ref_spectra_from_df
import math
from pathlib import Path
import pandas as pd
from itertools import product
from time import perf_counter
from multiprocessing.pool import ThreadPool
from multiprocessing import shared_memory
import numpy as np
import json
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import cupy as cp
import torch
import numba

assert torch.cuda.is_available()
assert cp.cuda.is_available()
assert numba.cuda.is_available()

In [71]:
from cudams.similarity.kernels import cosine_greedy_kernel

match_limit = 1024
max_peaks = 1024
batch_size = 2048
threshold = .75

kernel = compile_cuda_cosine_greedy_kernel(
    tolerance=.1,
    shift=0,
    mz_power=0,
    int_power=1,
    match_limit=match_limit,
    batch_size=batch_size,
)

In [72]:

from cudams.utils import get_spectra_batches

references, queries, batched_inputs = get_spectra_batches(
    batch_size=batch_size,
    max_peaks=max_peaks,
    max_pairs=(batch_size**2) * 64
)

100%|██████████| 32768/32768 [00:06<00:00, 5195.67it/s]


In [76]:
! rm -rf data/output
! mkdir -p data/output
for batch_i in tqdm(range(len(batched_inputs))):
    (rspec, rlen, rstart, rend), (qspec, qlen, qstart, qend) = batched_inputs[
        batch_i
    ]
    
    lens = cp.zeros((2, batch_size), "int32")
    lens[0, :len(rlen)] = cp.asarray(rlen)
    lens[1, :len(qlen)] = cp.asarray(qlen)
    rspec = cp.asarray(rspec)
    qspec = cp.asarray(qspec)
    out = cp.zeros((3, batch_size, batch_size), dtype="float32")
    
    kernel(rspec, qspec, lens, out)
    
    mask = out[0] >= threshold
    row, col = cp.nonzero(mask)
    score, matches, overflow = out[:, mask]
    rabs = rstart + row
    qabs = qstart + col
    cp.savez_compressed(f'data/output/{rstart}-{rend}-{qstart}-{qend}.npz', 
                        rabs=rabs, 
                        qabs=qabs, 
                        score=score, 
                        matches=matches,
                        overflow=overflow)

100%|██████████| 64/64 [00:28<00:00,  2.23it/s]


In [77]:
! du -hs data/output

5,1M	data/output


In [82]:
# Suppose we want to query these absolute query IDs, and sort their results

query = np.array([1, 42, 121, 100010])


In [89]:
q = cp.asarray(query)
for file in Path('data/output').glob('*.npz'):
    bunch = cp.load(file)
    qabs = bunch['qabs']
    interest = cp.isin(q, qabs)
    if interest.any():
        # We need to accumulate interests over all batches 
        # Concat them, sort them by `score` and return
        # cp.where(qabs == , )
    print(interest)
    break

[False False False False]
